In [3]:
import requests
import pandas as pd
import numpy as np
from scipy.stats import norm
from datetime import datetime, timedelta, timezone
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")

API_KEY = "67ffece4b2ae08.94077168"
BASE_URL = "https://eodhd.com/api/mp/unicornbay/options/contracts"

def black_scholes_call_price(S, K, T, r, sigma):
    if T <= 0 or sigma <= 0:
        return max(S - K, 0)
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

def get_current_price(ticker):
    url = f"https://eodhd.com/api/real-time/{ticker}.US?api_token={API_KEY}&fmt=json"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()
        return float(data.get("close", 0))
    except Exception as e:
        logging.warning(f"Failed to fetch current price for {ticker}: {e}")
        return 0

def fetch_filtered_options(symbol, exp_from, exp_to, strike_from, strike_to):
    params = {
        "api_token": API_KEY,
        "filter[underlying_symbol]": symbol,
        "filter[type]": "call",
        "filter[exp_date_from]": exp_from,
        "filter[exp_date_to]": exp_to,
        "filter[strike_from]": strike_from,
        "filter[strike_to]": strike_to,
        "sort": "exp_date",
        "page[limit]": 1000
    }

    try:
        response = requests.get(BASE_URL, params=params, timeout=10)
        response.raise_for_status()
        return response.json().get("data", [])
    except Exception as e:
        logging.warning(f"Error fetching options contracts for {symbol}: {e}")
        return []

def analyze_options_unicorn(
    tickers,
    option_type="call",
    days_until_exp=90,
    strike_pct=0.2,
    days_to_gain=30,
    stock_gain_pct=0.1,
    risk_free_rate=0.05
):
    today = datetime.now(timezone.utc)
    eval_date = today + timedelta(days=days_to_gain)
    exp_from = (today + timedelta(days=days_until_exp - 30)).strftime("%Y-%m-%d")
    exp_to = (today + timedelta(days=days_until_exp + 30)).strftime("%Y-%m-%d")

    results = []

    for ticker in tickers:
        try:
            current_price = get_current_price(ticker)
            if current_price <= 0:
                logging.info(f"Skipping {ticker}: invalid current price.")
                continue

            target_strike = current_price * (1 + strike_pct)
            lower = target_strike * 0.95
            upper = target_strike * 1.05

            options_data = fetch_filtered_options(ticker, exp_from, exp_to, lower, upper)
            if not options_data:
                logging.info(f"No options data returned for {ticker}")
                continue

            for opt in options_data:
                attr = opt.get("attributes", {})
                strike = attr.get("strike")
                exp_date_str = attr.get("exp_date")
                last_price = attr.get("last", 0)
                iv = attr.get("volatility", 0.3)

                if not strike or not exp_date_str:
                    continue

                exp_date = datetime.strptime(exp_date_str, "%Y-%m-%d").replace(tzinfo=timezone.utc)
                exp_days = (exp_date - today).days
                T_eval = max((exp_date - eval_date).days / 365, 0.0001)

                sim_stock = current_price * (1 + stock_gain_pct)
                est_value = black_scholes_call_price(sim_stock, strike, T_eval, risk_free_rate, iv)

                result = {
                    "Ticker": ticker,
                    "Expiration": exp_date_str,
                    "Strike": strike,
                    "% OTM/ITM": round((strike - current_price) / current_price * 100, 2),
                    "Current Stock": round(current_price, 2),
                    "Simulated Stock": round(sim_stock, 2),
                    "Current Premium": round(last_price, 2),
                    "Simulated Premium": round(est_value, 2),
                    "Days Until Expiration": exp_days,
                    "Days to Gain": days_to_gain,
                    "Stock Gain %": round(stock_gain_pct * 100, 2),
                    "% Gain": round(((est_value - last_price) / last_price) * 100, 2) if last_price else "NA",
                    "Implied Volatility": round(iv * 100, 2)
                }
                results.append(result)

        except Exception as e:
            logging.error(f"Error processing {ticker}: {e}")

    return pd.DataFrame(results)

# NEW: Helper function to run multiple parameter sets
def run_multiple_analyses(param_sets):
    combined_df = pd.DataFrame()
    for params in param_sets:
        label = params.pop("label", "")
        df = analyze_options_unicorn(**params)
        df["Run_Label"] = label
        combined_df = pd.concat([combined_df, df], ignore_index=True)
    return combined_df

# Run script
if __name__ == "__main__":
    param_sets = [
        {
            "tickers": ["AAPL", "MSFT"],
            "option_type": "call",
            "days_until_exp": 60,
            "strike_pct": 0.1,
            "days_to_gain": 20,
            "stock_gain_pct": 0.05,
            "label": "Short-Term Conservative"
        },
        {
            "tickers": ["AAPL", "AMD"],
            "option_type": "call",
            "days_until_exp": 120,
            "strike_pct": 0.3,
            "days_to_gain": 45,
            "stock_gain_pct": 0.15,
            "label": "Long-Term Aggressive"
        }
    ]

    combined_results = run_multiple_analyses(param_sets)
    display(combined_results)
    combined_results.to_csv("combined_unicorn_options.csv", index=False)


,Ticker,Expiration,Strike,% OTM/ITM,Current Stock,Simulated Stock,Current Premium,Simulated Premium,Days Until Expiration,Days to Gain,Stock Gain %,% Gain,Implied Volatility,Run_Label
0,AAPL,2025-06-20,225,6.64,210.99,221.54,2.16,3.00,34,20,5.0,38.88,24.95,Short-Term Conservative
1,AAPL,2025-06-20,230,9.01,210.99,221.54,1.35,1.49,34,20,5.0,10.02,24.83,Short-Term Conservative
2,AAPL,2025-06-20,235,11.38,210.99,221.54,0.79,0.69,34,20,5.0,-12.69,25.20,Short-Term Conservative
3,AAPL,2025-06-20,240,13.75,210.99,221.54,0.51,0.30,34,20,5.0,-40.37,25.75,Short-Term Conservative
4,AAPL,2025-06-27,225,6.64,210.99,221.54,2.70,4.02,41,20,5.0,48.90,24.89,Short-Term Conservative
5,AAPL,2025-06-27,230,9.01,210.99,221.54,1.73,2.30,41,20,5.0,32.75,24.62,Short-Term Conservative
6,AAPL,2025-06-27,235,11.38,210.99,221.54,1.03,1.26,41,20,5.0,21.98,24.80,Short-Term Conservative
7,AAPL,2025-06-27,240,13.75,210.99,221.54,0.65,0.67,41,20,5.0,2.42,25.20,Short-Term Conservative
8,AAPL,2025-07-18,225,6.64,210.99,221.54,4.15,6.47,62,20,5.0,55.99,24.93,Short-Term Conservative
9,AAPL,2025-07-18,230,9.01,210.99,221.54,2.86,4.44,62,20,5.0,55.11,24.53,Short-Term Conservative


In [27]:
import requests
import json

API_KEY = "67ffece4b2ae08.94077168"
BASE_URL = "https://eodhd.com/api/mp/unicornbay/options/contracts"

def fetch_specific_option(symbol, strike, exp_date, option_type="call"):
    params = {
        "api_token": API_KEY,
        "filter[underlying_symbol]": symbol,
        "filter[strike_eq]": strike,
        "filter[exp_date_eq]": exp_date,
        "filter[type]": option_type,
        "page[limit]": 1
    }

    try:
        response = requests.get(BASE_URL, params=params, timeout=10)
        response.raise_for_status()
        data = response.json().get("data", [])
        if not data:
            print(f"No option found for {symbol} {strike} {option_type.upper()} expiring {exp_date}")
            return

        contract = data[0].get("attributes", {})
        print(json.dumps(contract, indent=4))  # Pretty-print the full contract data

    except Exception as e:
        print(f"Error fetching option: {e}")

# Example usage
if __name__ == "__main__":
    fetch_specific_option(
        symbol="AAPL",
        strike=225,
        exp_date="2025-07-18",
        option_type="call"
    )


{
    "contract": "AAPL250718C00225000",
    "underlying_symbol": "AAPL",
    "exp_date": "2025-07-18",
    "expiration_type": "monthly",
    "type": "call",
    "strike": 225,
    "exchange": "NASDAQ",
    "currency": "USD",
    "open": 2.55,
    "high": 2.56,
    "low": 2.15,
    "last": 2.18,
    "last_size": 1,
    "change": -0.08,
    "pctchange": -3.54,
    "previous": 2.26,
    "previous_date": "2025-05-08",
    "bid": 2.14,
    "bid_date": "2025-05-09 19:59:59",
    "bid_size": 14,
    "ask": 2.2,
    "ask_date": "2025-05-09 19:59:59",
    "ask_size": 4,
    "moneyness": -0.13,
    "volume": 480,
    "volume_change": -1494,
    "volume_pctchange": -75.68,
    "open_interest": 13061,
    "open_interest_change": 633,
    "open_interest_pctchange": 5.09,
    "volatility": 0.2763,
    "volatility_change": -0.0074,
    "volatility_pctchange": -2.62,
    "theoretical": 2.18,
    "delta": 0.178321,
    "gamma": 0.010934,
    "theta": -0.048553,
    "vega": 0.22509,
    "rho": 0.06281,

In [39]:
from polygon import RESTClient
from dataclasses import asdict
import json

# Your Polygon API key
API_KEY = "3CMDTNYo0AX8XqZ9Su6xKZXhdxFSN83P"

# AAPL 225 Call expiring 2025-07-18
underlying_symbol = "AAPL"
contract_symbol = "O:AAPL250718C00225000"

def get_option_snapshot(api_key, ticker, option_symbol):
    try:
        client = RESTClient(api_key)
        snapshot = client.get_snapshot_option(ticker, option_symbol)

        # Convert to dict and pretty print
        snapshot_dict = asdict(snapshot)
        print("\n=== Full Snapshot ===")
        print(json.dumps(snapshot_dict, indent=2))

        # Print key fields (attribute-based access)
        print("\n--- Key Metrics ---")
        print(f"Break-even Price: {snapshot.break_even_price}")
        print(f"Implied Volatility: {snapshot.implied_volatility}")
        print(f"Open Interest: {snapshot.open_interest}")

        if snapshot.greeks:
            print("\nGreeks:")
            print(f"  Delta: {snapshot.greeks.delta}")
            print(f"  Gamma: {snapshot.greeks.gamma}")
            print(f"  Theta: {snapshot.greeks.theta}")
            print(f"  Vega: {snapshot.greeks.vega}")
        else:
            print("No Greeks available.")

        if snapshot.last_trade:
            print(f"\nLast Trade Price: {snapshot.last_trade.price}")
        else:
            print("No last trade data available.")

        if snapshot.last_quote:
            print(f"Bid: {snapshot.last_quote.bid} | Ask: {snapshot.last_quote.ask}")
        else:
            print("No quote data available.")

        if snapshot.underlying_asset:
            print(f"Underlying Price: {snapshot.underlying_asset.price}")
        else:
            print("No underlying data available.")

    except Exception as e:
        print("Error retrieving option snapshot:", e)

# Run the function
get_option_snapshot(API_KEY, underlying_symbol, contract_symbol)



=== Full Snapshot ===
{
  "break_even_price": null,
  "day": {
    "change": 1.52,
    "change_percent": 69.725,
    "close": 3.7,
    "high": 4.94,
    "last_updated": 1747022400000000000,
    "low": 3.38,
    "open": 4.05,
    "previous_close": 2.18,
    "volume": 914,
    "vwap": 3.9336
  },
  "details": {
    "contract_type": "call",
    "exercise_style": "american",
    "expiration_date": "2025-07-18",
    "shares_per_contract": 100,
    "strike_price": 225,
    "ticker": "O:AAPL250718C00225000"
  },
  "greeks": {
    "delta": 0.2824241075659986,
    "gamma": 0.01531474699283917,
    "theta": -0.06265016421402456,
    "vega": 0.33404555544673825
  },
  "implied_volatility": 0.2495599452298498,
  "last_quote": null,
  "last_trade": null,
  "open_interest": 13199,
  "underlying_asset": {
    "change_to_break_even": null,
    "last_updated": null,
    "price": null,
    "value": null,
    "ticker": "AAPL",
    "timeframe": null
  },
  "fair_market_value": null
}

--- Key Metrics ---

In [41]:
from polygon import RESTClient
from dataclasses import asdict
import json

# Your Polygon.io API key
API_KEY = "3CMDTNYo0AX8XqZ9Su6xKZXhdxFSN83P"

# Example contract: AAPL 225C 2025-07-18
contract_symbol = "O:AAPL250718C00225000"

def get_option_contract_info(api_key, contract_symbol):
    try:
        client = RESTClient(api_key)
        contract = client.get_options_contract(contract_symbol)

        # Convert dataclass to dictionary for display
        contract_dict = asdict(contract)
        print("\n=== Contract Metadata ===")
        print(json.dumps(contract_dict, indent=2))

        # Optional: print key info nicely
        details = contract.results
        print("\n--- Key Attributes ---")
        print(f"Ticker: {details.ticker}")
        print(f"Contract Type: {details.contract_type}")
        print(f"Exercise Style: {details.exercise_style}")
        print(f"Strike Price: {details.strike_price}")
        print(f"Expiration Date: {details.expiration_date}")
        print(f"Shares Per Contract: {details.shares_per_contract}")
        print(f"Underlying Ticker: {details.underlying_ticker}")
        print(f"Primary Exchange: {details.primary_exchange}")

        if details.additional_underlyings:
            print("\nAdditional Underlyings:")
            for item in details.additional_underlyings:
                print(f" - {item.amount} units of {item.underlying} ({item.type})")

    except Exception as e:
        print("Error retrieving contract info:", e)

# Run it
get_option_contract_info(API_KEY, contract_symbol)



=== Contract Metadata ===
{
  "additional_underlyings": null,
  "cfi": "OCASPS",
  "contract_type": "call",
  "correction": null,
  "exercise_style": "american",
  "expiration_date": "2025-07-18",
  "primary_exchange": "BATO",
  "shares_per_contract": 100,
  "strike_price": 225,
  "ticker": "O:AAPL250718C00225000",
  "underlying_ticker": "AAPL"
}
Error retrieving contract info: 'OptionsContract' object has no attribute 'results'
